In [33]:
from pathlib import Path
p = Path('.')
subfiles = [x for x in p.iterdir() if x.is_file()]

In [34]:
pending_rename = [x for x in subfiles if x.match("*_*")]
for file_ in pending_rename:
    new_name = file_.name.replace("_", "-").replace(".html", ".md")
    file_.rename(new_name)

In [94]:
markdown_files = [x for x in p.iterdir() if x.is_file() and x.match("*.md")]

import re
import fileinput
import sys

pattern1 = re.compile(r".*({% *geshi([^%]*)%})(.*)", re.MULTILINE|re.DOTALL)
pattern2 = re.compile(r"(.*)({% *endgeshi[^%]*%}).*", re.MULTILINE|re.DOTALL)

def ungeshify(file_):
    file_name = file_.resolve()
    with fileinput.FileInput(file_name, inplace=True) as f:
        for line in f:
            new_line = line
            m = pattern1.search(line)
            if m:
                new_line = f"```"
                programming_language = m.group(2)
                if programming_language:
                    programming_language = programming_language.strip().replace("'", "").replace('"', "")
                    new_line = f"{new_line}{programming_language}\n"
                maybe_content = m.group(3)
                if maybe_content and maybe_content.strip():
                    new_line = f"{new_line}{maybe_content}"
            m = pattern2.search(new_line)
            if m:
                maybe_content = m.group(1)
                if maybe_content and maybe_content.strip():
                    maybe_content = f"{maybe_content}\n"
                else:
                    maybe_content = ""
                new_line = f"{maybe_content}```\n"
            sys.stdout.write(new_line)

for markdown in markdown_files:
    ungeshify(markdown)